# 0.Name & Z-ID (5 pts)
Mounika Thudi \
Z1948596 \
Assignment 7

# 1b. [CSCI 503] Download & Extract Files (30 pts)

In [1]:
import nest_asyncio

nest_asyncio.apply()


In [2]:
import asyncio
import aiohttp
import shutil
from pathlib import Path
import os
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import threading
import numpy as np
import glob

In [3]:
async def download_site(session, url):
    async with session.get(url) as response:
        
        path_of_file = Path(url)
        name_of_file= path_of_file.name
        flag = False

    async with session.get(url) as response:
        if os.path.exists(name_of_file):
            print(f'path exists......{name_of_file}')
            flag = True
        
        if not flag:
            with open(name_of_file, 'wb') as file:
                async for data in response.content.iter_chunked(1024):
                    file.write(data)
    return name_of_file
            

async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        return await asyncio.gather(*tasks, return_exceptions=True)

if __name__ == "__main__":
    sites = [
        "http://faculty.cs.niu.edu/~dakoop/cs503-2022sp/a7/unemp-1970.zip",
        "http://faculty.cs.niu.edu/~dakoop/cs503-2022sp/a7/unemp-1980.zip",
        "http://faculty.cs.niu.edu/~dakoop/cs503-2022sp/a7/unemp-1990.zip",
        "http://faculty.cs.niu.edu/~dakoop/cs503-2022sp/a7/unemp-2000.zip",
        "http://faculty.cs.niu.edu/~dakoop/cs503-2022sp/a7/unemp-2010.zip",
        "http://faculty.cs.niu.edu/~dakoop/cs503-2022sp/a7/unemp-2020.zip",
    ] * 1
    downloaded_files=asyncio.get_event_loop().run_until_complete(download_all_sites(sites))
    print(downloaded_files)
    
    cd = os.getcwd()
    decades = [1970, 1980, 1990, 2000, 2010, 2020]         
    for decade in decades:
        #shutil.rmtree(f'unemp-{decade}')
        if(not os.path.exists(os.path.join(cd, f'unemp-{decade}'))):
            shutil.unpack_archive(f'unemp-{decade}.zip')
            print(f'zip file unemp-{decade} extracted successfully')
        else:
            print(f'zip file unemp-{decade} extracted already')

path exists......unemp-1980.zip
path exists......unemp-1970.zip
path exists......unemp-1990.zip
path exists......unemp-2000.zip
path exists......unemp-2010.zip
path exists......unemp-2020.zip
['unemp-1970.zip', 'unemp-1980.zip', 'unemp-1990.zip', 'unemp-2000.zip', 'unemp-2010.zip', 'unemp-2020.zip']
zip file unemp-1970 extracted already
zip file unemp-1980 extracted already
zip file unemp-1990 extracted already
zip file unemp-2000 extracted already
zip file unemp-2010 extracted already
zip file unemp-2020 extracted already


# 2. Find Matching Files (10 pts)

In [4]:
current_directory=os. getcwd()
all_csv_files = glob.glob(os.path.join(current_directory,'**', '*.csv'), recursive = True)
print(all_csv_files)

['/home/z1948596/unemp-2010/employment.csv', '/home/z1948596/unemp-1970/unemp.csv', '/home/z1948596/unemp-2000/unemployment.csv', '/home/z1948596/unemp-2020/unemp.csv', '/home/z1948596/unemp-1990/employment.csv', '/home/z1948596/unemp-1980/unemp.csv']


# 3. Use Threads to Process Files (30 pts)

In [5]:
my_lock = threading.Lock()
dataframe_list= []
def process_csv(filelist):
    with my_lock:
        
        df = pd.read_csv(filelist)
        df = df[['COUNTY','YEAR','LABOR_FORCE','EMPLOYED','UNEMPLOYED_NUMBER']]
        df['COUNTY'] = df['COUNTY'].str.removesuffix(" COUNTY")
        df['COUNTY'] = df['COUNTY'].str.upper()
        newdf = df[df.COUNTY.isin(['DEKALB', 'KANE', 'BOONE', 'MCHENRY', 'WINNEBAGO', 'OGLE', 'LEE','KENDALL'])]
        newdf=newdf.assign(RATE = newdf['UNEMPLOYED_NUMBER'] / newdf['LABOR_FORCE'])
        dataframe_list.append(newdf)
        
        
with ThreadPoolExecutor(max_workers=6) as executor:
    all_data = executor.map(process_csv, all_csv_files, timeout=60)


# concatenate, into single dataframe
concat_data = pd.concat(dataframe_list,ignore_index=0) 

In [6]:
counties_no_suffix = ['DEKALB', 'KANE', 'BOONE', 'MCHENRY', 'WINNEBAGO', 'OGLE', 'LEE', 'KENDALL']  
for cnty in counties_no_suffix:
    concat_data[concat_data['COUNTY'].str.contains(cnty)].to_csv(cnty+'.csv.gz')

In [7]:
 for c in sorted(counties_no_suffix):
    cdf = pd.read_csv(f'{c}.csv.gz')
    print(c, cdf.RATE.mean())

BOONE 0.08099938784155404
DEKALB 0.05766694572727062
KANE 0.06465744592318969
KENDALL 0.055213620478529436
LEE 0.06341724493660843
MCHENRY 0.05963539940389765
OGLE 0.06660581534993117
WINNEBAGO 0.07757288231143362
